<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/analyse_LA_mobility_poverty_time_series_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [59]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.54 s (started: 2021-04-15 22:36:29 +00:00)


In [60]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u282-b08-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
time: 5.06 s (started: 2021-04-15 22:36:32 +00:00)


In [61]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 6.39 ms (started: 2021-04-15 22:36:37 +00:00)


In [62]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 9.57 ms (started: 2021-04-15 22:36:37 +00:00)


In [63]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 231 ms (started: 2021-04-15 22:36:37 +00:00)


In [64]:
#sc.stop()

time: 1.02 ms (started: 2021-04-15 22:36:38 +00:00)


In [65]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 2.78 ms (started: 2021-04-15 22:36:38 +00:00)


In [66]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'la'

time: 1.33 ms (started: 2021-04-15 22:36:38 +00:00)


In [67]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [68]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [69]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 1.7 ms (started: 2021-04-15 22:36:45 +00:00)


In [70]:
df_soc.createOrReplaceTempView('clean_la')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_la WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|                              1|                              1|
|060374034022|2020-02-07T00:00:...|         155|                          24|                             12|                              4|
|060375326041|2020-02-07T00:00:...|          31|                           4|                              6|                              2|
|060375544031|2020-02-07T00:00:...|         130|                          27|                             16|                             10|
|06037

In [71]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)

time: 11.2 ms (started: 2021-04-15 22:36:45 +00:00)


In [72]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|                              1|                              1|        13.333333333333334|       2.2222222222222223|       2.2222222222222223|
|060374034022|2020-02-07T00:00:...|         155|                          24|                             12|       

In [73]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)
 |-- part_time_work_percentage: double (nullable = true)
 |-- full_time_work_percentage: double (nullable = true)

time: 3.95 ms (started: 2021-04-15 22:36:46 +00:00)


In [74]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 12 ms (started: 2021-04-15 22:36:46 +00:00)




---


demographic data






---



In [75]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'la.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|060373104003|     1286|17.884914463452567| 69.51788491446345|
|060590865023|     2545|22.789783889980352| 85.34381139489194|
|060376510024|     1285|               0.0|42.490272373540854|
|060376511011|     2708| 7.745398773006134| 50.70162481536189|
|060590994023|      672|12.969283276450511| 38.54166666666667|
|060590995094|      342| 17.83625730994152| 77.48538011695906|
|060376511012|     2483|2.6178010471204187|50.302053966975436|
|060376512011|      859|2.8846153846153846| 82.42142025611176|
|060376512012|     1179| 1.441899915182358|  74.1306191687871|
|060376512014|      989|2.4266936299292214| 70.57633973710819|
|060376512211|     3373|  9.95684340320592| 40.61666172546695|
|060590995092|      938| 5.756929637526652|  84.9680170575693|
|060590995095|      576|14.583333333333334|           7

In [76]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|poverty_percentage|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|                              1|                              1|        13.333333333333334|       2.2222222222222223|       2.2222222222222223|15.450643776824036|
|060374034022|2020-02-07T00:00:...|     

In [77]:

#grouped_df_mob_demo.show()

time: 695 µs (started: 2021-04-15 22:36:46 +00:00)


In [78]:
# Count number of rows
#spark.sql('SELECT COUNT(cbg) FROM demographic_mobility').show()

time: 1.33 ms (started: 2021-04-15 22:36:46 +00:00)


In [79]:
# DIVIDING THE POPULATION IN 3 GROUPS 

# df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 30')
# grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
# grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


# #grouped_df_mob_demo_high_class.printSchema()


# df_mob_demo_middle_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage >= 30 AND poverty_percentage < 70 ')
# grouped_df_mob_demo_middle_class = df_mob_demo_middle_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
# grouped_df_mob_demo_middle_class.createOrReplaceTempView('mob_demo_middle_class')


# df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage >= 70')
# grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
# grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

# #df_mob_demo_merhed = spark.sql('SELECT h.*, m.')


# grouped_df_mob_demo_combined = spark.sql("""

# SELECT h.date,h.completely_home_percentage as high_comp_home_perc,
# m.completely_home_percentage as middle_comp_home_perc,
# l.completely_home_percentage as low_comp_home_perc  
# FROM mob_demo_high_class h INNER JOIN mob_demo_middle_class m ON h.date = m.date INNER JOIN mob_demo_low_class l ON m.date = l.date

# """)



time: 16.1 ms (started: 2021-04-15 22:36:46 +00:00)


In [80]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


#grouped_df_mob_demo_high_class.printSchema()


# df_mob_demo_middle_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage >= 30 AND poverty_percentage < 70 ')
# grouped_df_mob_demo_middle_class = df_mob_demo_middle_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
# grouped_df_mob_demo_middle_class.createOrReplaceTempView('mob_demo_middle_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

#df_mob_demo_merhed = spark.sql('SELECT h.*, m.')


grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.completely_home_percentage as high_comp_home_perc,
l.completely_home_percentage as low_comp_home_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")


time: 138 ms (started: 2021-04-15 22:36:46 +00:00)


In [81]:
#Sort by poverty
# df_mob_demo = spark.sql('SELECT * FROM demographic_mobility ORDER BY CAST(poverty_percentage AS float) DESC')
# df_mob_demo.createOrReplaceTempView('demographic_mobility')
# df_mob_demo.show()

time: 1.5 ms (started: 2021-04-15 22:36:47 +00:00)


In [82]:
#grouped_df_mob_demo_combined.show()


time: 1.26 ms (started: 2021-04-15 22:36:47 +00:00)


In [83]:
#count = grouped_df_mob_demo_combined.count()

time: 1.25 ms (started: 2021-04-15 22:36:47 +00:00)


In [84]:
print("{}".format(count))

<function count at 0x7f1608c97440>
time: 2.18 ms (started: 2021-04-15 22:36:47 +00:00)


In [85]:
grouped_df_mob_demo_combined_pandas = grouped_df_mob_demo_combined.toPandas()
grouped_df_mob_demo_combined_pandas

,date,high_comp_home_perc,low_comp_home_perc
0,2020-07-02T00:00:00-07:00,36.119294,38.879257
1,2020-11-13T00:00:00-08:00,29.819945,34.897599
2,2020-06-02T00:00:00-07:00,42.603093,45.551986
3,2020-06-04T00:00:00-07:00,36.198207,41.083774
4,2020-10-29T00:00:00-07:00,29.936463,30.359378
...,...,...,...
361,2020-04-28T00:00:00-07:00,43.718724,25.330862
362,2020-05-26T00:00:00-07:00,38.940051,39.605983
363,2020-06-13T00:00:00-07:00,35.180490,35.412627
364,2020-08-28T00:00:00-07:00,32.588181,32.356150


time: 3min 54s (started: 2021-04-15 22:36:47 +00:00)


---
graphs 

---

In [86]:
base = alt.Chart(grouped_df_mob_demo_combined_pandas.reset_index()).encode(x='date')

alt.layer(
    base.mark_line(color='blue').encode(y='high_comp_home_perc'),
    # base.mark_line(color='red').encode(y='middle_comp_home_perc'),
    base.mark_line(color='green').encode(y='low_comp_home_perc')

)

alt.LayerChart(...)

time: 113 ms (started: 2021-04-15 22:40:41 +00:00)


In [87]:
# alt.Chart(result_mean_df).mark_bar(size=30).encode(
#     alt.X('poverty_percentage:N'),
#     alt.Y('mean_completely_home_percentage'),
#     color = alt.Color('color',scale=None),
#     tooltip=[alt.Tooltip('poverty_percentage'),
#              alt.Tooltip('mean_completely_home_percentage')]
# ).properties(width=400, height=400).interactive()

time: 3.18 ms (started: 2021-04-15 22:40:42 +00:00)


In [88]:
# #
# alt.Chart(result_mean_df).mark_point().encode(
#     alt.X('poverty_percentage:N'),
#     alt.Y('mean_completely_home_percentage'),
#     color = alt.Color('color',scale=None),
#     tooltip=[alt.Tooltip('poverty_percentage'),
#              alt.Tooltip('mean_completely_home_percentage')]
# ).properties(width=400, height=400).interactive()



time: 2.37 ms (started: 2021-04-15 22:40:42 +00:00)


In [89]:
# result_df['poverty_percentage'].astype(float).dtype

time: 1.07 ms (started: 2021-04-15 22:40:42 +00:00)


In [90]:
# #
# alt.Chart(result_mean_df).mark_point().encode(
#     alt.X('poverty_percentage:N'),
#     alt.Y('mean_full_time_work_percentage'),
#     color = alt.Color('color',scale=None),
#     tooltip=[alt.Tooltip('poverty_percentage'),
#              alt.Tooltip('mean_full_time_work_percentage')]
# ).properties(width=400, height=400).interactive()

time: 2.53 ms (started: 2021-04-15 22:40:42 +00:00)


In [91]:
# #
# alt.Chart(result_mean_df).mark_point().encode(
#     alt.X('poverty_percentage:N'),
#     alt.Y('mean_part_time_work_percentage'),
#     color = alt.Color('color',scale=None),
#     tooltip=[alt.Tooltip('poverty_percentage'),
#              alt.Tooltip('mean_part_time_work_percentage')]
# ).properties(width=400, height=400).interactive()

time: 2.59 ms (started: 2021-04-15 22:40:42 +00:00)
